# Tool Calling com LangChain
<img src="https://python.langchain.com/assets/images/tool_call-8d4a8b18e90cacd03f62e94071eceace.png">

Neste notebook, vamos aprender como usar modelos de chat para chamar ferramentas (tools) usando a LangChain. Este é um conceito poderoso que permite que modelos de linguagem interajam com funções definidas.

## O que é Tool Calling?

Tool Calling permite que um modelo de chat responda a um prompt chamando uma "ferramenta". É importante notar que:
- O modelo não executa diretamente a ação
- O modelo apenas gera os argumentos para a ferramenta
- A execução real da ferramenta é controlada pelo usuário

## Pré-requisitos
Antes de começarmos, vamos instalar as bibliotecas necessárias:

# Definindo Schemas de Ferramentas

Existem várias maneiras de definir schemas para nossas ferramentas. Vamos explorar três abordagens diferentes:
1. Usando funções Python
2. Usando classes Pydantic
3. Usando TypedDict

## 1. Funções Python
Primeiro, vamos ver como definir ferramentas usando funções Python simples:

In [ ]:
import os
from dotenv import load_dotenv  

load_dotenv(dotenv_path=".env")
OPENAI_MODEL_NAME = os.getenv('OPENAI_MODEL_NAME')


In [57]:
import os
from dotenv import load_dotenv  
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

@tool
def add(a: int, b: int) -> int:
    """Add two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a * b

# Registrar as ferramentas
tools = [add, multiply]
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Vincular o LLM às ferramentas
llm_with_tools = llm.bind_tools(tools)

# Pergunta ao LLM
query = "Quanto é 3 * 13?"
response = llm_with_tools.invoke(query)

# 🔹 Verificando se houve chamadas de ferramentas
if response.tool_calls:
    for tool_call in response.tool_calls:
        tool_name = tool_call["name"]  # Nome da ferramenta que foi chamada
        tool_args = tool_call["args"]  # Argumentos que o LLM passou para a ferramenta

        # 🔹 Executa a ferramenta correta com os argumentos extraídos
        if tool_name == "multiply":
            result = multiply.invoke(tool_args)
        elif tool_name == "add":
            result = add.invoke(tool_args)
        
        print(f"Resultado da ferramenta `{tool_name}`: {result}")




Resultado da ferramenta `multiply`: 39


In [66]:
import os
import json
from dotenv import load_dotenv  
from langchain_core.tools import StructuredTool
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from langchain_core.messages import HumanMessage

# Carregar variáveis de ambiente
load_dotenv(dotenv_path=".env")
OPENAI_MODEL_NAME = os.getenv('OPENAI_MODEL_NAME')

# Definição do esquema de entrada
class AddInput(BaseModel):
    number1: int = Field(..., description="Primeiro número inteiro")
    number2: int = Field(..., description="Segundo número inteiro")

# Função da ferramenta
def add(number1: float, number2: float) -> float:
    "Essa função faz a somatória de um number1 mais um number2"
    return number1 + number2

# Criar ferramenta usando StructuredTool
add_tool = StructuredTool(
    name="add",
    func=add,
    description="Essa função realiza a soma de 2 números.",
    args_schema=AddInput  # Define explicitamente os parâmetros
)

# Criar LLM e vinculá-lo à ferramenta
llm = ChatOpenAI(model="gpt-4o", temperature=0)
llm_with_tool = llm.bind_tools([add_tool])

# Consulta ao LLM
query = "Olá, quanto é 2 + 2?"


messages = [HumanMessage(content=query)]
response = llm_with_tool.invoke([HumanMessage(content=query)])


messages.append(response)
# Verificando chamadas de ferramenta
if response.tool_calls:
    for tool_call in response.tool_calls:
        tool_name = tool_call["name"] 
        tool_args = tool_call["args"]  
        print(f"🔹 Tool chamada: {tool_name}")
        print(f"🔹 Argumentos recebidos: {tool_args}")

        if tool_name == "add":
            result = add_tool.invoke(tool_args)  
        
        print(f"✅ Resultado da ferramenta `{tool_name}`: {result}")

print(messages)

🔹 Tool chamada: add
🔹 Argumentos recebidos: {'number1': 2, 'number2': 2}
✅ Resultado da ferramenta `add`: 4
[HumanMessage(content='Olá, quanto é 2 + 2?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_wqHDDamt9pis4XTrNR4WptMT', 'function': {'arguments': '{"number1":2,"number2":2}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 71, 'total_tokens': 91, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f9f4fb6dbf', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d0847346-a63b-4d99-81a2-221c625cd277-0', tool_calls=[{'name': 'add', 'args': {'number1': 2, 'number2': 2}, 'id': 'call_wqHDDamt9pis4XTrNR4WptMT', 'type': '